In [1]:
import random
import time
import matplotlib.pyplot as plt
import os

In [2]:
# Fungsi untuk membaca puzzle dari file
def read_puzzles_from_file(filename):
    with open(filename, 'r') as file:
        content = file.read()
        puzzles = [list(map(int, line.split())) for line in content.strip().split('\n\n')]
    return puzzles

# Fungsi untuk menghasilkan Sudoku dasar
def generate_base_sudoku():
    while True:
        puzzle = [[0 for _ in range(9)] for _ in range(9)]
        numbers = list(range(1, 10))
        random.shuffle(numbers)

        if fill_sudoku(puzzle, numbers):
            return puzzle

# Fungsi untuk mengisi Sudoku
def fill_sudoku(puzzle, numbers, visualize=False):
    row, col = find_empty_location(puzzle)
    if row is None:
        return True

    for num in numbers:
        if is_valid(puzzle, num, row, col):
            puzzle[row][col] = num
            
            if visualize:
                print(f"Menempatkan {num} di ({row}, {col})")
                print_sudoku(puzzle)
                plt.pause(0.5)  # Delay untuk visualisasi

            if fill_sudoku(puzzle, numbers, visualize):
                return True

            puzzle[row][col] = 0
            
            if visualize:
                print(f"Menghapus {num} dari ({row}, {col})")
                print_sudoku(puzzle)
                plt.pause(0.5)  # Delay untuk visualisasi

    return False

# Fungsi untuk menemukan lokasi kosong
def find_empty_location(puzzle):
    for row in range(9):
        for col in range(9):
            if puzzle[row][col] == 0:
                return row, col
    return None, None

# Fungsi untuk memeriksa validitas angka
def is_valid(puzzle, num, row, col):
    for i in range(9):
        if puzzle[row][i] == num:
            return False

    for i in range(9):
        if puzzle[i][col] == num:
            return False

    box_row = (row // 3) * 3
    box_col = (col // 3) * 3
    for i in range(3):
        for j in range(3):
            if puzzle[box_row + i][box_col + j] == num:
                return False
    return True

# Fungsi untuk mencetak Sudoku
def print_sudoku(puzzle):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - ")
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print(" | ", end="")
            print(puzzle[i][j], end=" ")
        print()

In [3]:
# Kelas untuk CSP (Constraint Satisfaction Problem)
class CSP:
    def __init__(self, variables, Domains, constraints,n_langkah):
        self.variables = variables
        self.domains = Domains
        self.constraints = constraints
        self.solution = None
        self.n_langkah =0

    def solve(self, visualize=False):
        assignment = {}
        self.solution = self.backtrack(assignment, visualize)
        if self.solution is None:
            print("Puzzle Sudoku ini tidak dapat diselesaikan.")
            exit()
        return self.solution

    def backtrack(self, assignment, visualize):
        if len(assignment) == len(self.variables):
            return assignment

        var = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(var, assignment):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                if visualize:
                    self.n_langkah=self.n_langkah + 1
                    print(f"Menempatkan {value} di {var}")
                    print_sudoku(self.construct_solution(assignment))

                result = self.backtrack(assignment, visualize)
                if result is not None:
                    return result
                del assignment[var]
                if visualize:
                    self.n_langkah=self.n_langkah + 1
                    print(f"Menghapus {value} dari {var}")
                    print_sudoku(self.construct_solution(assignment))

        return None

    def select_unassigned_variable(self, assignment):
        unassigned_vars = [var for var in self.variables if var not in assignment]
        return min(unassigned_vars, key=lambda var: len(self.domains[var]))

    def order_domain_values(self, var, assignment):
        return self.domains[var]

    def is_consistent(self, var, value, assignment):
        for constraint_var in self.constraints[var]:
            if constraint_var in assignment and assignment[constraint_var] == value:
                return False
        return True

    def construct_solution(self, assignment):
        # Membuat matriks Sudoku dari assignment
        solution = [[0 for _ in range(9)] for _ in range(9)]
        for (i, j), value in assignment.items():
            solution[i][j] = value
        return solution

In [4]:
# Interaksi utama
# Mendapatkan direktori saat ini
current_directory = os.path.dirname(os.path.abspath("__file__"))
input_file_path = os.path.join(current_directory, 'InputTest.txt')

puzzles = read_puzzles_from_file(input_file_path)
print(f"Tersedia {len(puzzles)} puzzle. Pilih puzzle (1-{len(puzzles)}):")
for index, puzzle in enumerate(puzzles, start=1):
    print(f"\nPuzzle {index}:")
    print_sudoku([puzzle[i:i + 9] for i in range(0, 81, 9)])

puzzle_choice = int(input("Masukkan nomor puzzle yang ingin dipilih: ")) - 1
if 0 <= puzzle_choice < len(puzzles):
    puzzle = [puzzles[puzzle_choice][i:i + 9] for i in range(0, 81, 9)]
else:
    print("Pilihan tidak valid. Keluar.")
    exit()

print("\nPuzzle Sudoku yang dipilih:")
print_sudoku(puzzle)

variables = [(i, j) for i in range(9) for j in range(9)]
Domains = {var: set(range(1, 10)) if puzzle[var[0]][var[1]] == 0
                        else {puzzle[var[0]][var[1]]} for var in variables}

# Fungsi untuk menambahkan batasan
def add_constraint(var):
    constraints[var] = []
    for i in range(9):
        if i != var[0]:
            constraints[var].append((i, var[1]))
        if i != var[1]:
            constraints[var].append((var[0], i))
    sub_i, sub_j = var[0] // 3, var[1] // 3
    for i in range(sub_i * 3, (sub_i + 1) * 3):
        for j in range(sub_j * 3, (sub_j + 1) * 3):
            if (i, j) != var:
                constraints[var].append((i, j))

constraints = {}
for i in range(9):
    for j in range(9):
        add_constraint((i, j))

print("Tampilkan solusi dengan visualisasi? (1 = ya)")
while True:
    show_solution = input("> ")
    if show_solution == "1":
        plt.ion()  # Mengaktifkan mode interaktif untuk visualisasi
        print('*' * 7, 'Solusi', '*' * 7)
        csp = CSP(variables, Domains, constraints,0)
        sol = csp.solve(visualize=True)

        solution = [[0 for i in range(9)] for i in range(9)]
        for (i, j), value in sol.items():
            solution[i][j] = value

        print("Solusi akhir:")
        print_sudoku(solution)
        print("Banyak Langkah Yang dikerjakan : ",csp.n_langkah)
        plt.ioff()  # Mematikan mode interaktif
        break

Tersedia 3 puzzle. Pilih puzzle (1-3):

Puzzle 1:
1 0 0  | 0 0 0  | 0 0 0 
0 0 0  | 0 0 0  | 0 0 0 
0 0 0  | 0 0 0  | 0 0 0 
- - - - - - - - - - - 
0 0 0  | 0 0 0  | 0 0 0 
0 0 0  | 0 0 0  | 0 0 0 
0 0 0  | 0 0 0  | 0 0 0 
- - - - - - - - - - - 
0 0 0  | 0 0 0  | 0 0 0 
0 0 0  | 0 0 0  | 0 0 0 
0 0 0  | 0 0 0  | 0 0 0 

Puzzle 2:
1 0 0  | 0 0 0  | 0 0 0 
0 2 0  | 0 0 0  | 0 0 0 
0 0 3  | 0 0 0  | 0 0 0 
- - - - - - - - - - - 
0 0 0  | 4 0 0  | 0 0 0 
0 0 0  | 0 5 0  | 0 0 0 
0 0 0  | 0 0 6  | 0 0 0 
- - - - - - - - - - - 
0 0 0  | 0 0 0  | 7 0 0 
0 0 0  | 0 0 0  | 0 8 0 
0 0 0  | 0 0 0  | 0 0 9 

Puzzle 3:
5 0 0  | 7 0 0  | 9 2 0 
0 8 0  | 0 2 6  | 5 3 4 
0 0 6  | 5 0 0  | 0 8 1 
- - - - - - - - - - - 
0 5 7  | 4 0 8  | 0 0 0 
4 1 0  | 0 6 9  | 0 0 7 
0 0 9  | 0 5 7  | 0 0 0 
- - - - - - - - - - - 
0 4 5  | 0 0 3  | 0 0 0 
1 0 0  | 6 0 2  | 0 0 0 
6 9 2  | 0 0 5  | 0 7 3 

Puzzle Sudoku yang dipilih:
5 0 0  | 7 0 0  | 9 2 0 
0 8 0  | 0 2 6  | 5 3 4 
0 0 6  | 5 0 0  | 0 8 1 
- - - - - -